### Import Libraries

In [2]:
import pandas as pd
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.vectorstores import FAISS
import pickle
import requests
import logging
import re
from datetime import datetime

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

### Load Data

In [3]:
# Path to the JSONL file
file_path = './data/test_data.jsonl'

# Load and process the data
logging.info("Loading dataset from file.")
data = pd.read_json(file_path, lines=True)
logging.info(f"Dataset loaded with {len(data)} rows and {len(data.columns)} columns.")

2024-12-20 13:59:06,140 - INFO - Loading dataset from file.
2024-12-20 13:59:06,216 - INFO - Dataset loaded with 10 rows and 56 columns.


### Embedding Data

In [4]:
# Columns to embed
columns_to_embed = [
    'nama', 'sid', 'sid_tsat', 'produk', 'layanan', 'alamat', 'project_nama', 'no_wo', 
    'jenis_workorder', 'jenis_order', 'status_nodelink', 
    'customer', 'customer_direct', 'channeling', 'segmen', 'start_kontrak', 'end_kontrak'
]

# Combine columns into a single text field per row
data['combined_text'] = data.apply(
    lambda row: " \n ".join(
        f"{col}: {row[col]}" for col in columns_to_embed if pd.notnull(row[col]) and row[col] != "-"
    ),
    axis=1
)

# Initialize embedding model
embedding_model_name = "all-MiniLM-L6-v2"
logging.info(f"Initializing embedding model: {embedding_model_name}")
embedding_model = SentenceTransformerEmbeddings(model_name=embedding_model_name)

2024-12-20 13:59:06,243 - INFO - Initializing embedding model: all-MiniLM-L6-v2
C:\Users\andidprastyo\AppData\Local\Temp\ipykernel_3664\137906283.py:19: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = SentenceTransformerEmbeddings(model_name=embedding_model_name)
c:\Users\andidprastyo\Documents\Github\llm-query\.conda\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-12-20 13:59:16,528 - INFO - Use pytorch device_name: cpu
2024-12-20 13:59:16,529 - INFO - Load pr

### Create Vector Store

In [5]:
# Create separate vector stores for each column
vector_stores = {}

logging.info("Creating vector stores for each column.")
for column in columns_to_embed:
    logging.info(f"Processing column: {column}")
    # Drop rows with null or placeholder values for the current column
    valid_rows = data[data[column].notnull() & (data[column] != "-")]
    texts = valid_rows[column].tolist()
    metadata = [{"index": idx, "column": column} for idx in valid_rows.index]
    
    # Create FAISS vector store for the column
    # Convert all texts to strings to accommodate int values
    texts = [str(text) for text in texts]
    vector_store = FAISS.from_texts(texts, embedding_model, metadatas=metadata)
    vector_stores[column] = vector_store
    logging.info(f"Vector store created for column: {column}, with {len(texts)} entries.")

# Save the vector stores and the original data
save_data = {
    "vector_stores": vector_stores,
    "data": data
}

vector_store_file = "vector_stores_by_column.pkl"
with open(vector_store_file, "wb") as f:
    pickle.dump(save_data, f)
logging.info(f"Vector stores saved to '{vector_store_file}'.")


2024-12-20 13:59:20,002 - INFO - Creating vector stores for each column.
2024-12-20 13:59:20,003 - INFO - Processing column: nama
2024-12-20 13:59:20,253 - INFO - Loading faiss with AVX512 support.
2024-12-20 13:59:20,253 - INFO - Could not load library with AVX512 support due to:
ModuleNotFoundError("No module named 'faiss.swigfaiss_avx512'")
2024-12-20 13:59:20,255 - INFO - Loading faiss with AVX2 support.
2024-12-20 13:59:20,359 - INFO - Successfully loaded faiss with AVX2 support.
2024-12-20 13:59:20,381 - INFO - Vector store created for column: nama, with 10 entries.
2024-12-20 13:59:20,382 - INFO - Processing column: sid
2024-12-20 13:59:20,429 - INFO - Vector store created for column: sid, with 10 entries.
2024-12-20 13:59:20,430 - INFO - Processing column: sid_tsat
2024-12-20 13:59:20,524 - INFO - Vector store created for column: sid_tsat, with 6 entries.
2024-12-20 13:59:20,526 - INFO - Processing column: produk
2024-12-20 13:59:20,573 - INFO - Vector store created for column:

### Helper Functions (Querying Ollama, Context Processing)

In [6]:
def query_ollama(prompt):
    """Query the Ollama API with a given prompt."""
    logging.info("Sending query to Ollama API.")
    url = "http://localhost:11434/api/chat"
    headers = {"Content-Type": "application/json"}
    payload = {
        "model": "llama3.2:1b",
        "stream": False,
        "messages": [
            {
                "role": "user",
                "content": prompt
            }
        ],
        "options": {
            "temperature": 0.5,
            "top_p": 0.95,
            "max_tokens": 150
        }
    }

    try:
        response = requests.post(url, headers=headers, json=payload)
        response.raise_for_status()
        result = response.json()
        if 'message' in result and 'content' in result['message']:
            return result['message']['content'].strip()
        else:
            logging.warning("Unexpected response format from Ollama API.")
            return "Error: Message or content not found in response."
    except Exception as e:
        logging.error(f"Error querying Ollama API: {e}")
        return None

def retrieve_context(query, vector_stores, target_column=None, data=None):
    """Retrieve relevant context from the FAISS vector store for specific columns."""
    if target_column not in vector_stores:
        logging.error(f"No vector store found for column '{target_column}'.")
        return None

    docs = []
    
    # Handle exact match columns (IDs and dates)
    if target_column in ['sid', 'sid_tsat', 'no_wo', 'start_kontrak', 'end_kontrak']:
        # Clean and extract the search value
        if ":" in query:
            search_value = query.split(":")[-1].strip()
        else:
            # Extract the last part of the query
            search_value = query.split()[-1].strip()
        
        # Remove any trailing punctuation or spaces
        search_value = search_value.rstrip('?.,!')

        exact_matches = []
        
        if target_column in ['start_kontrak', 'end_kontrak']:
            # First try exact date match (YYYY-MM-DD)
            if re.match(r'20\d{2}-\d{2}-\d{2}', search_value):
                exact_matches = data[data[target_column] == search_value].index.tolist()
            
            # Then try year match (YYYY)
            elif re.match(r'20\d{2}', search_value):
                exact_matches = data[data[target_column].str.startswith(search_value, na=False)].index.tolist()
            
            # Handle relative year terms
            else:
                current_year = pd.Timestamp.now().year
                relative_terms = {
                    'this year': str(current_year),
                    'next year': str(current_year + 1),
                    'previous year': str(current_year - 1),
                    'last year': str(current_year - 1)
                }
                
                for term, year in relative_terms.items():
                    if term.lower() in query.lower():
                        exact_matches = data[data[target_column].str.startswith(year, na=False)].index.tolist()
                        break
                
        elif target_column in ['sid', 'sid_tsat']:
            try:
                # Try numeric conversion
                numeric_value = pd.to_numeric(search_value)
                exact_matches = data[data[target_column] == numeric_value].index.tolist()
            except ValueError:
                # Fallback to string comparison
                exact_matches = data[data[target_column].astype(str) == search_value].index.tolist()
        
        else:  # no_wo
            # Use string comparison
            exact_matches = data[data[target_column].astype(str) == search_value].index.tolist()
        
        if exact_matches:
            for idx in exact_matches:
                docs.append(type('Document', (), {
                    'page_content': str(data.loc[idx, target_column]),
                    'metadata': {'index': idx, 'column': target_column}
                })())
            logging.info(f"Found exact match(es) for {target_column}: {search_value}")
        else:
            logging.info(f"No exact match found for {target_column}: {search_value}")
            
            # Only if no exact matches are found, use vector similarity as fallback
            retriever = vector_stores[target_column].as_retriever(
                search_type="similarity",
                search_kwargs={"k": 5}
            )
            docs = retriever.get_relevant_documents(query)
            
            if docs:
                logging.info(f"Using similarity search as fallback for {target_column}")

    # Remove duplicates while preserving order
    seen_indices = set()
    unique_docs = []
    for doc in docs:
        idx = doc.metadata['index']
        if idx not in seen_indices:
            seen_indices.add(idx)
            unique_docs.append(doc)
    docs = unique_docs[:10]

    # Log retrieved documents
    logging.info(f"Retrieved {len(docs)} documents for column '{target_column}'.")
    for i, doc in enumerate(docs):
        logging.info(f"Doc {i + 1}: Content: {doc.page_content}, Metadata: {doc.metadata}")
    
    return docs

def ask_dataset(query, vector_stores, data, target_column):
    docs = retrieve_context(query, vector_stores, target_column, data)
    if not docs:
        return "Error: No relevant context found for the query."

    contexts = []
    for doc in docs:
        index = doc.metadata.get('index')
        if index in data.index:
            original_data = data.loc[index]
            contexts.append({
                'relevance_index': len(contexts) + 1,
                'data': {col: original_data[col] for col in data.columns 
                        if pd.notnull(original_data[col]) and original_data[col] != "-"}
            })

    formatted_context = "\n\n".join([
        f"Result {ctx['relevance_index']}:\n" + 
        "\n".join([f"{k}: {v}" for k, v in ctx['data'].items()])
        for ctx in contexts
    ])

    prompt = f"""Context:
{formatted_context}

Question: {query}

Instructions:
1. Analyze the results and prioritize contracts ending in (years if years else the specified year).
2. Use exact year matches over similarity ones.
3. Include and verify all relevant date information.
4. Preserve data formatting in your response.
5. Provide clear conclusions based on the extracted data.

Provide a concise answer based on this context."""

    response = query_ollama(prompt)
    return response


### Testing

In [7]:
# Load the saved vector stores
with open(vector_store_file, "rb") as f:
    saved_data = pickle.load(f)
    vector_stores = saved_data["vector_stores"]
    data = saved_data["data"]

# Example query - can search by any column
query = "Which contracts has sid : 5480001690001"
target_column = "sid"  # Can be any column from the data
response = ask_dataset(query, vector_stores, data, target_column)
print("Response from the dataset:\n", response)

2024-12-20 13:59:22,358 - INFO - Found exact match(es) for sid: 5480001690001
2024-12-20 13:59:22,360 - INFO - Retrieved 1 documents for column 'sid'.
2024-12-20 13:59:22,361 - INFO - Doc 1: Content: 5480001690001, Metadata: {'index': 8, 'column': 'sid'}
2024-12-20 13:59:22,366 - INFO - Sending query to Ollama API.


Response from the dataset:
 Berikut adalah hasil pencarian untuk sid: 5480001690001:

- Start kontrak: 2020
- End kontrak: 2022
- Jenis workorder: Aktivasi
- Status nodelink: Operasional
- Customer Direct: TELKOM DES
- Channeling: INDIRECT DES
- Segmen: BANKING

Dari hasil pencarian, ditemukan satu kontrak yang memenuhi kondisi yang ditanyakan.


In [8]:
query = "Which contracts end in 2025"
target_column = "end_kontrak"  # Can be any column from the data
response = ask_dataset(query, vector_stores, data, target_column)
print("Response from the dataset:\n", response)

2024-12-20 13:59:52,129 - INFO - Found exact match(es) for end_kontrak: 2025
2024-12-20 13:59:52,130 - INFO - Retrieved 2 documents for column 'end_kontrak'.
2024-12-20 13:59:52,132 - INFO - Doc 1: Content: 2025-04-23, Metadata: {'index': 1, 'column': 'end_kontrak'}
2024-12-20 13:59:52,133 - INFO - Doc 2: Content: 2025-04-23, Metadata: {'index': 2, 'column': 'end_kontrak'}
2024-12-20 13:59:52,138 - INFO - Sending query to Ollama API.


Response from the dataset:
 Based on the provided results, the contracts that end in 2025 are:

- Result 1: id: 6000, nama: MANAGE SERVICE RADIO IP BANK IBK INDONESIA
- Result 2: id: 16632, nama: ROUTER MIKROTIK CCR BANK IBK INDONESIA

These two contracts end in the year 2025.
